(update description)

<img src="images/mesh_material.png" alt="Mesh and Material" width="300"/>
Fig.1 An octree mesh generated from 3D image-based model of concrete

In [1]:
%%bash

cat <<EOL > setup_env.sh
source ~/env_MPI/bin/activate
export PYTHONPATH=~/env_MPI/lib/python3.11/site-packages:\$PYTHONPATH
module load python3/3.11.0
module load openmpi/4.0.5

export work_dir=~/PCG_MPI_Solver
cd \$work_dir

export n_meshparts=8

EOL


In [2]:
%%bash
source setup_env.sh

export model_name=concrete
export scratch_path=$work_dir/data
export input_model_path=$scratch_path/$model_name.zip

python3 src/data/read_input_model.py $work_dir $model_name $scratch_path $input_model_path


>creating directories..


>extracting files from concrete


>success!


>elements:  124693


>nodes:     208316 


>dofs:      624948


>total runtime: 2.06 sec


In [3]:
%%bash
source setup_env.sh

python3 src/solver/run_metis.py $n_meshparts



>loading model data..


>generating indices for 8 mesh parts..


>success!


>total runtime: 0.57 sec


<img src="images/mesh_partition.png" alt="Mesh Partition" width="300"/>
Fig.2 Mesh partitions shown in different colours

In [4]:
%%bash
source setup_env.sh

export n_cores=4   
mpiexec -np $n_cores --map-by numa python3 src/solver/partition_mesh.py $n_meshparts 0


>loading model data..


>partitioning mesh into 8 parts using 4 cores..


>success!


>total runtime: 3.05 sec


In [5]:
%%bash
source setup_env.sh

python3 -c "
from src.utils.file_operations import exportz

TimeHistoryParam={'ExportFlag': True, # export results
                  'ExportFrmRate' : 1,
                  'ExportFrms' : [],
                  'PlotFlag' : False,
                  'TimeStepDelta': [0,1],
                  'ExportVars': 'U'} #D U PS PE GS GE

SolverParam={'Tol' : 1e-7,
             'MaxIter' : 10000}

GlobSettings = {'TimeHistoryParam' : TimeHistoryParam,
                'SolverParam' : SolverParam}
exportz('__pycache__/GlobSettings.zpkl', GlobSettings)
"

In [6]:
%%bash
source setup_env.sh

mpiexec -np $n_meshparts --map-by numa python3 src/solver/pcg_solver.py 1 0


>loading partitioned data..


>running parallel pcg solver with 8 cores..


>exporting results..


>success!


>file read time:     0.2 sec 


>calculation time:   11.5 sec


>communication time: 1.0 sec


>---------------------------


>total runtime:      12.6 sec 


In [7]:
%%bash
source setup_env.sh

mpiexec -np 1 python3 src/data/export_vtk.py 1 "U" "Full"


>loading model and result data..


>exporting vtk files..


>success!


>total runtime: 2.27 sec


<img src="images/mesh_deformation.png" alt="Mesh Deformation" width="350"/>
Fig.3 Mesh deformation (scaled 1000 times for illustration)